In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
import yfinance as yf

# Read in the CSV file containing the stock tickers
df = pd.read_csv('h.csv')

# Extract the list of stock tickers from the 'symbol' column
ticker_list = list(df['symbol'].values)

# Create a Tickers object with the list of tickers
tickers = yf.Tickers(ticker_list)

# Create an empty list to store the business summaries
summaries = []

# Iterate over each ticker and retrieve the business summary
for ticker in ticker_list:
    try:
        summary = tickers.tickers[ticker].info['longBusinessSummary']
        summaries.append(summary)
    except:
        summaries.append(None)

# Add the business summaries as a new column in the DataFrame
df['Business Summary'] = summaries

# Save the updated DataFrame to a new CSV file
df.to_csv('h_sum.csv', index=False)

In [ ]:
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.6 MB/s eta 0:00:00


In [ ]:
import openai
import pandas as pd
import time

openai.api_key = " "  # Replace with your actual API key

def classify_company(summary):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o",  # Ensure you're using the correct model identifier
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"Based on the following business summary, provide a 5-word brief explanation of what the company does:\n\n{summary}\n\nExplanation:"}
            ],
            max_tokens=20,
            temperature=0.7,
        )
        explanation = response.choices[0].message['content'].strip()
        return explanation
    except Exception as e:
        print(f"An error occurred: {e}")
        return ""

# Read the CSV file
df = pd.read_csv("sheet.csv")  # Replace "sheet.csv" with your actual CSV file name

# Set the rate limit (e.g., 60 requests per minute)
rate_limit = 60
interval = 60 / rate_limit

# Initialize a list to store the company classifications
classification_list = []

# Iterate over each business summary and classify the company with rate limiting
for summary in df["Business Summary"]:
    classification = classify_company(summary)
    classification_list.append(classification)

    # Sleep for the specified interval to maintain the rate limit
    time.sleep(interval)

# Add the company classifications as a new column in the DataFrame
df["Company Classification"] = classification_list

# Reorder the columns to have Company Classification first, followed by Ticker and Business Summary
df = df[["Company Classification", "Ticker", "Business Summary"]]

# Save the updated DataFrame to a new CSV file
df.to_csv("classified_company_data.csv", index=False)

# Print the updated DataFrame
print(df)


                              Company Classification Ticker  \
0           Life sciences and diagnostics solutions.      A   
1              Produces and sells aluminum products.     AA   
2    Provides diverse educational services globally.   AACG   
3        Acquires fintech companies through mergers.   AACI   
4        Special purpose acquisition company (SPAC).   AACT   
..                                               ...    ...   
814       Leases diversified commercial real estate.    BNL   
815                Develops drugs for CNS disorders.   BNOX   
816         Develops cancer therapy selection tests.    BNR   
817     Insurance and reinsurance services provider.   BNRE   
818         Develops thermal energy storage systems.   BNRG   

                                      Business Summary  
0    Agilent Technologies, Inc. provides applicatio...  
1    Alcoa Corporation, together with its subsidiar...  
2    ATA Creativity Global, together with its subsi...  
3    Armada Acq